In [13]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from datetime import datetime
from sklearn.preprocessing import power_transform

Trabajaremos con los dos dataset a la par, limpiandolos y procesandolos.

Cada transformacion y limpieza se ha realizado dadas las observaciones y analisis del EDA.

In [2]:
# Carga de datos
df_ratings = pd.read_parquet('../raw/data_model/ratings_train.parquet')
df_tags = pd.read_parquet('../raw/data_model/tags_train.parquet')

In [3]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000175 entries, 0 to 16000174
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     float64
 1   movieId    int64  
 2   rating     float64
 3   timestamp  float64
 4   title      object 
 5   genres     object 
dtypes: float64(3), int64(1), object(2)
memory usage: 732.4+ MB


In [3]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9367814 entries, 0 to 9367813
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
 3   tag        object 
dtypes: float64(1), int64(2), object(1)
memory usage: 285.9+ MB


Limpieza de datos: nulos y duplicados

In [5]:
# df_ratings, revisando nulos
df_ratings.isna().sum()

userId       415
movieId        0
rating       415
timestamp    415
title          0
genres         0
dtype: int64

In [4]:
# Los valores nulos de df_ratings los modificaremos con el valor 0
def imputer_dfRatings (df_ratings):

    df_ratings_copy = df_ratings.copy()
    # Instancia del imputer
    imp_0 = SimpleImputer(strategy="constant", fill_value=0)
    df_ratings_copy = imp_0.fit_transform(df_ratings_copy)

    #Convertir a dataframe nuevamente
    df_ratings_copy = pd.DataFrame(df_ratings_copy, columns=imp_0.get_feature_names_out())
    return df_ratings_copy

In [5]:
df_ratings= imputer_dfRatings(df_ratings)# llamada a funcion de limpieza de nulos 

In [6]:
# df_tags, revisando nulos
df_tags.isna().sum()

movieId      0
tagId        0
relevance    0
tag          0
dtype: int64

Referente a los valores nulos, hemos modificado los NaN a 0, dado que hay peliculas en el df_ratings que no han sido valoradas por ningun usuario.

415 peliculas no han sido vistas ni valoradas

In [9]:
df_ratings[df_ratings['userId']==0]

,userId,movieId,rating,timestamp,title,genres
20615,0,113949,0,0,Aaron Loves Angela (1975),Comedy|Drama|Romance|Thriller
80864,0,117610,0,0,Doctor at Large (1957),Comedy|Romance
91125,0,127851,0,0,Incredible Rocky Mountain Race (1977),Drama
118641,0,117178,0,0,Amira & Sam (2014),Comedy|Drama|Romance
127055,0,121055,0,0,Harvest (1937),Drama
...,...,...,...,...,...,...
15881848,0,118183,0,0,Dying Room Only (1973),Horror|Mystery|Thriller
15903193,0,105796,0,0,Back in the Saddle (Back in the Saddle Again) ...,Action|Drama|Western
15931978,0,121875,0,0,Submarine (1928),Action|Drama
15955176,0,115376,0,0,"Black Sleep, The (1956)",Horror|Sci-Fi


In [10]:
# df_ratings, revisando duplicados en titulos (no queremos el mismo titulo mas de 1 vez)
df_ratings.nunique() # Comprobamos los valores duplicados por columnas

userId         138494
movieId         26255
rating             11
timestamp    12685451
title           26255
genres           1328
dtype: int64

In [11]:
df_tags.nunique() # Comprobamos duplicados en tags, en este caso no es necesario hacer nada

movieId      10381
tagId         1128
relevance     4000
tag           1128
dtype: int64

Los dataset se encuentran correctamente con respecto a los duplicados.
- En df_ratings no tenemos ningun titulo igual repetido.
- En df_tags tenemos la misma cantidad de tag como de su id.

Transformaciones de variables e ingenieria de caracteristicas en df_ratings

In [9]:
# Transformamos las variables del dataset df_ratings
# Timestamp, convertimos a datetime (referente a fecha de valoracion)
def timestamp_to_datetime_dfRatings(df_ratings):
    df_ratings_copy = df_ratings.copy()
    # Convertimos el timestamp a datetime, luego aplicamos el formato año/mes/dia devolviendo un string
    df_ratings_copy['date_rating'] = df_ratings_copy['timestamp'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
    # Transformamos a Date el String
    df_ratings_copy['date_rating'] = df_ratings_copy['date_rating'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    # Eliminamos variable timestamp
    df_ratings_copy.drop('timestamp', axis=1, inplace=True)

    return df_ratings_copy

In [10]:
df_ratings=timestamp_to_datetime_dfRatings(df_ratings) # Transforma timestamp a datetime (las fechas no valoradas se veran asi:"1970-01-01")

In [15]:
# Transformamos las variables del dataset df_ratings
# title, Extraemos el año de publicacion de cada pelicula en una variable nueva
def launchYear_title_dfRatings(df_ratings):
    df_ratings_copy = df_ratings.copy()
    # Extraemos con expresiones regulares los años del title, rellenamos los nulos con 0 y casteamos a int
    df_ratings_copy['launch_year'] = df_ratings_copy['title'].str.extract(r'\((\d{4})\)', expand=False).fillna('0').astype(int)
    # Eliminamos el año y cualquier contenido entre paréntesis en la columna 'title'
    df_ratings_copy['title'] = df_ratings_copy['title'].apply(lambda x: x.split(' (')[0])
    return df_ratings_copy

In [16]:
df_ratings=launchYear_title_dfRatings(df_ratings) # Extrae año del titulo

In [19]:
# Transformamos las variables del dataset df_ratings
# Convertimos userId a entero
def int_userId_dfRatings(df_ratings):
    df_ratings_copy=df_ratings.copy()
    # Convertir 'userId' a int
    df_ratings_copy['userId'] = pd.to_numeric(df_ratings_copy['userId'], errors='coerce').astype('int64')
    return df_ratings_copy

In [20]:
df_ratings=int_userId_dfRatings(df_ratings)# Convierte userId a entero

In [27]:
# Transformamos las variables del dataset df_ratings
# Extraemos la longitud de cada title y lo añadimos como nueva variable
def lenTitle_dfRatings(df_ratings):
    df_ratings_copy=df_ratings.copy()
    #Extraemos la longitud de cada uno de los titulos
    df_ratings_copy['len_title']=df_ratings_copy['title'].apply(lambda x: len(x))

    return df_ratings_copy

In [28]:
df_ratings=lenTitle_dfRatings(df_ratings) # Extraemos longitud del titulo

In [32]:
# Transformamos las variables del dataset df_ratings
# Extraemos  el genero principal de cada movieId (se considera principal al primero que aparece en genres antes del '|')
def mainGenre_dfRatings(df_ratings):
    df_ratings_copy=df_ratings.copy()
    # Extraemos el primer genero de la columna genres y lo almacenamos en una nueva variable
    df_ratings_copy['main_genre']=df_ratings_copy['genres'].apply(lambda x: x.split('|')[0])
    return df_ratings_copy


In [33]:
df_ratings=mainGenre_dfRatings(df_ratings) #Extraemos genero principal

In [38]:
# Guardamos cambios de datos procesados (esto nos permite un mayor control ante nuevos cambios)
df_ratings.to_parquet('../raw/data_model/ratings_train_processV0.1.parquet', index=False)

Transformaciones de variables e ingenieria de caracteristicas en df_tags

In [4]:
df_tags

,movieId,tagId,relevance,tag
0,2961,1024,0.27850,thought-provoking
1,91371,1031,0.01175,tom clancy
2,992,380,0.02700,fascism
3,545,164,0.04000,british
4,1845,411,0.05750,french
...,...,...,...,...
9367809,2904,216,0.08225,classic car
9367810,25769,415,0.39150,fun movie
9367811,1594,972,0.40350,storytelling
9367812,238,632,0.01025,marvel


In [10]:
# Tranformamos distribucion relevance a normal, se encuentra sesgada
def distribution_relevance_dfTags(df_tags):
    df_tags_copy = df_tags.copy()
    df_tags_copy['relevance']=power_transform(df_tags_copy[['relevance']], method='box-cox')
    return df_tags_copy

In [11]:
df_tags=distribution_relevance_dfTags(df_tags) # Realizamos transformacion de distribucion relevance

In [16]:
# Guardamos cambios de datos procesados (esto nos permite un mayor control ante nuevos cambios)
df_tags.to_parquet('../raw/data_model/tags_train_processV0.1.parquet', index=False)